<a href="https://colab.research.google.com/github/skywalker0803r/c620/blob/main/usage_example(colab)V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/skywalker0803r/c620.git
!pip install autorch > log.txt
!pip install optuna > log.txt

Cloning into 'c620'...
remote: Enumerating objects: 2202, done.
remote: Counting objects: 100% (1020/1020), done.
remote: Compressing objects: 100% (562/562), done.
remote: Total 2202 (delta 663), reused 687 (delta 449), pack-reused 1182
Receiving objects: 100% (2202/2202), 53.96 MiB | 22.72 MiB/s, done.
Resolving deltas: 100% (1441/1441), done.


# import

In [2]:
from c620.FV2 import AllSystem
from c620.configV2 import config
import joblib
import numpy as np
import pandas as pd
import os
os.chdir('/content/c620')
print(os.getcwd())
!git pull

/content/c620
Already up to date.


# demo data

In [3]:
demo = joblib.load('./data/demo.pkl')
print(demo.keys())
icg_input = demo['icg_input']
c620_feed = demo['c620_feed']
t651_feed = demo['t651_feed']
icg_input.head()

dict_keys(['icg_input', 'c620_feed', 't651_feed', 'c620_op', 'c620_wt', 'c660_op', 'c660_wt', 'c670_op', 'c670_wt'])


,Simulation Case Conditions_Feed Rate_Feed from V615 Btm_m3/hr,Simulation Case Conditions_Feed Rate_Feed from V615 Btm NA_wt%,Simulation Case Conditions_Feed Rate_Feed from V615 Btm Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from V615 Btm Toluene_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist_m3/hr,Simulation Case Conditions_Feed Rate_Feed from C820 Dist NA_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist Toluene_wt%,Simulation Case Conditions_Feed Rate_Feed from T651_m3/hr,Simulation Case Conditions_Feed Rate_Feed from T651 NA_wt%,Simulation Case Conditions_Feed Rate_Feed from T651 Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from T651 Toluene_wt%,Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%,Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw,Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr,Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw,Tatoray Stripper C620 Operation_Specifications_Spec 1 : Receiver Temp_oC
002-002,163.0,1.356,10.321001,40.887009,15.0,2.338977,74.302254,17.467825,41.0,0.4524,44.244236,44.786339,70,980,0.25,10,36.0


In [4]:
icg_input[['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw']]

,Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw
002-002,980


In [5]:
icg_input[['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%']]

,Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%
002-002,70


In [6]:
icg_input[['Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw']]

,Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw
002-002,10


# 1.試算模式

In [7]:
f = joblib.load('/content/c620/model/allsystem.pkl')

In [8]:
demo['icg_input']

,Simulation Case Conditions_Feed Rate_Feed from V615 Btm_m3/hr,Simulation Case Conditions_Feed Rate_Feed from V615 Btm NA_wt%,Simulation Case Conditions_Feed Rate_Feed from V615 Btm Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from V615 Btm Toluene_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist_m3/hr,Simulation Case Conditions_Feed Rate_Feed from C820 Dist NA_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from C820 Dist Toluene_wt%,Simulation Case Conditions_Feed Rate_Feed from T651_m3/hr,Simulation Case Conditions_Feed Rate_Feed from T651 NA_wt%,Simulation Case Conditions_Feed Rate_Feed from T651 Benzene_wt%,Simulation Case Conditions_Feed Rate_Feed from T651 Toluene_wt%,Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%,Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw,Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr,Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw,Tatoray Stripper C620 Operation_Specifications_Spec 1 : Receiver Temp_oC
002-002,163.0,1.356,10.321001,40.887009,15.0,2.338977,74.302254,17.467825,41.0,0.4524,44.244236,44.786339,70,980,0.25,10,36.0


In [9]:
c620_wt,c620_op,c660_wt,c660_op,c670_wt,c670_op = f.inference(demo['icg_input'],demo['c620_feed'],demo['t651_feed'])

# 檢查三項約束條件

In [10]:
c620_wt[['Tatoray Stripper C620 Operation_Sidedraw Production Rate and Composition_Benzene_wt%']]

,Tatoray Stripper C620 Operation_Sidedraw Production Rate and Composition_Benzene_wt%
002-002,71.350981


In [11]:
na_idx = [1,2,3,4,5,6,8,9,11,13,14,15,20,22,29] 
c660_wt.filter(regex='Side').filter(regex='wt%').iloc[:,na_idx].sum(axis=1).values[0]*10000

964.9365272378022

In [12]:
c660_wt['Benzene Column C660 Operation_Sidedraw (Benzene )Production Rate and Composition_Toluene_wt%'].values[0]*10000

9.149712539088709

In [13]:
c620_wt.iloc[:,:41].sum(axis=1)

002-002    100.0
dtype: float64

In [14]:
c660_wt.iloc[:,:41].sum(axis=1)

002-002    100.0
dtype: float64

In [15]:
c670_wt.iloc[:,:41].sum(axis=1)

002-002    100.0
dtype: float64

# 推薦模式

In [16]:
c620_wt2,c620_op2,c660_wt2,c660_op2,c670_wt2,c670_op2,bz_error,nainbz_error,tol_error = f.recommend(demo['icg_input'],demo['c620_feed'],demo['t651_feed'])

[I 2021-04-26 08:45:19,984] A new study created in memory with name: no-name-b79cff91-63a0-47aa-82cc-8037bfc7fe61
[I 2021-04-26 08:45:20,057] Trial 0 finished with value: 19.059033197692514 and parameters: {'Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 14 (Control)_oC': 175.43462677791433, 'Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 34 (Control)_oC': 193.0905004780978, 'Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr': 3.9958407635814965}. Best is trial 0 with value: 19.059033197692514.
[I 2021-04-26 08:45:20,146] Trial 1 finished with value: 3.529847082849484 and parameters: {'Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 14 (Control)_oC': 176.80734885379798, 'Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 34 (Control)_oC': 193.73477685535798, 'Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr': 0.6143970041860757}. Best is trial 1 with value: 3.529847082849484.

bz_error: 0.00014741498958414923
nainbz_error: 12.417995034525461
tol_error: 0.028256596194497163
dist_rate 1.0005982913260736
nainbz: 967.5820049654745


In [17]:
bz_error,nainbz_error,tol_error

(0.00014741498958414923, 12.417995034525461, 0.028256596194497163)

# 計算delta調幅

In [28]:
(c620_op2-c620_op).iloc[:,-2:]

,Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 14 (Control)_oC,Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 34 (Control)_oC
002-002,-0.373634,0.705435


In [29]:
(c660_op2-c660_op).iloc[:,-2:]

,Benzene Column C660 Operation_Column Temp Profile_C660 Tray 6 (SD & Control)_oC,Benzene Column C660 Operation_Column Temp Profile_C660 Tray 23 (Control)_oC
002-002,0.0452,0.606887


In [30]:
(c670_op2-c670_op).iloc[:,-2:]

,Toluene Column C670 Operation_Column Temp Profile_C670 Tray 24 (Control)_oC,Toluene Column C670 Operation_Column Temp Profile_C670 Btm Temp (Control)_oC
002-002,0.002151,0.043945


# 檢查三項約束條件

In [21]:
c620_wt2[['Tatoray Stripper C620 Operation_Sidedraw Production Rate and Composition_Benzene_wt%']]

,Tatoray Stripper C620 Operation_Sidedraw Production Rate and Composition_Benzene_wt%
002-002,70.000147


In [22]:
na_idx = [1,2,3,4,5,6,8,9,11,13,14,15,20,22,29] 
c660_wt2.filter(regex='Side').filter(regex='wt%').iloc[:,na_idx].sum(axis=1).values[0]*10000

967.5820049654745

In [23]:
c660_wt2['Benzene Column C660 Operation_Sidedraw (Benzene )Production Rate and Composition_Toluene_wt%'].values[0]*10000

10.028256596194497

In [24]:
c620_wt2.iloc[:,:41].sum(axis=1)

002-002    100.0
dtype: float64

In [25]:
c660_wt2.iloc[:,:41].sum(axis=1)

002-002    100.0
dtype: float64

In [26]:
c670_wt2.iloc[:,:41].sum(axis=1)

002-002    100.0
dtype: float64